In [1]:
include("../src/ProcessSequence.jl")

Main.ProcessSequence

In [27]:
project_dir = dirname(@__DIR__)

using JSON: parse

project_json = parse(read(joinpath(project_dir, "project.json"), String))

output_dir = joinpath(project_dir, "output")

input_dir = joinpath(project_dir, "input")

reference_dir = joinpath(input_dir, "reference")

baseline_vcf = joinpath(input_dir, "high_confidence_HG002_4.1_GRCh38/HG002_GRCh38_1_22_v4.1_draft_benchmark.vcf.gz")

calls_vcf = joinpath(output_dir, "process_germ_dna_HG002_cat/find_variant/strelka/results/variants/variants.vcf.gz")

rtg_sdf_reference_dir = joinpath(input_dir, "GRCh38.sdf")

benchmark_output_dir = joinpath(output_dir, "process_germ_dna_HG002_cat/benchmark")

rtg_executable = "/opt/rtg/rtg-tools-3.11/rtg"

"/opt/rtg/rtg-tools-3.11/rtg"

### Configure vcfeval

In [5]:
run(`$rtg_executable`)

RTG has a facility to automatically send basic usage information to Real
Time Genomics. This does not contain confidential information such as
command-line parameters or dataset contents.


Automatic usage reporting disabled.


Initial configuration complete.  Advanced user configuration is
available by editing settings in rtg.cfg and users of bash may wish to
enable bash completion, see /opt/rtg/rtg-tools-3.11/scripts/rtg-bash-completion




Usage: rtg COMMAND [OPTION]...
       rtg RTG_MEM=16G COMMAND [OPTION]...  (e.g. to set maximum memory use to
	             	16 GB)

Type 'rtg help COMMAND' for help on a specific command.
The following commands are available:

Data formatting:
	format       	convert sequence data files to SDF
	sdf2fasta    	convert SDF to FASTA
	sdf2fastq    	convert SDF to FASTQ
	sdf2sam      	convert SDF to SAM/BAM
	fastqtrim    	trim reads in FASTQ files

Simulation:
	genomesim    	generate simulated genome sequence
	cgsim        	generate simulated reads from a sequence
	readsim      	generate simulated reads from a sequence
	popsim       	generate a VCF containing simulated population variants
	samplesim    	generate a VCF containing a genotype simulated from a population
	childsim     	generate a VCF containing a genotype simulated as a child of two
	             	parents
	denovosim    	generate a VCF containing a derived genotype containing de novo
	             	variants
	pedsamplesim 	generat

LoadError: failed process: Process(`/opt/rtg/rtg-tools-3.11/rtg`, ProcessExited(1)) [1]


### Download Real Time Genomics prepared GRCH38 files
vcfeval requires reference files in RTG formats. RTG makes versions of these files for the popular reference genomes such as GRCh38, which will be downloaded below.

In [25]:
if ispath(rtg_sdf_reference_dir)
    
    println("Skipping download because RTG GRCh38 data directory already exists:\n $rtg_sdf_reference_dir")

else

    run(pipeline(
            `wget -q -P ../input/ https://s3.amazonaws.com/rtg-datasets/references/GRCh38.sdf.zip`,
            `unzip -d ../input/ ../input/GRCh38.sdf.zip`,
            `rm ../input/GRCh38.sdf.zip`,
            ))
end

Skipping download because RTG GRCh38 data directory already exists:
 /home/jovyan/ProcessSequence.jl/input/GRCh38.sdf


### Run vcfeval
This command comapares the VCF you generated (calls) to the highly confident VCF generated by Genome in a Bottle (baseline).

In [28]:
run(`$rtg_executable vcfeval 
    --baseline=$baseline_vcf
    --calls=$calls_vcf
    --template=$rtg_sdf_reference_dir
    --output=$benchmark_output_dir
    --threads=1
    `)

Error: Unable to find file: /home/jovyan/ProcessSequence.jl/input/GRCh38.sdf/mainIndex (No such file or directory) part of SDF: /home/jovyan/ProcessSequence.jl/input/GRCh38.sdf


LoadError: failed process: Process(`/opt/rtg/rtg-tools-3.11/rtg vcfeval --baseline=/home/jovyan/ProcessSequence.jl/input/high_confidence_HG002_4.1_GRCh38/HG002_GRCh38_1_22_v4.1_draft_benchmark.vcf.gz --calls=/home/jovyan/ProcessSequence.jl/output/process_germ_dna_HG002_cat/find_variant/strelka/results/variants/variants.vcf.gz --template=/home/jovyan/ProcessSequence.jl/input/GRCh38.sdf --output=/home/jovyan/ProcessSequence.jl/output/process_germ_dna_HG002_cat/benchmark --threads=2`, ProcessExited(1)) [1]


### See results
Read more about how to interpret vcfeval results [here](https://cdn.rawgit.com/RealTimeGenomics/rtg-tools/master/installer/resources/tools/RTGOperationsManual/rtg_command_reference.html#vcfeval).

In [ ]:
vcfeval_summary_file = open(joinpath(benchmark_output_dirput_dir, "summary.txt"))

vcfeval_summary = read(vcfeval_summary_file)

vcfeval_summary